# Python trick collection

Inspired Ned Batchelder when he posted a python 27 version of `__cmp__` on twitter, made me think it's about time to start a small collection of neat 1-line python tricks.
 More inspirations are welcome - just submit an issue or pull request.

## Number magic

Here's the inspiration (thanks Ned):

<blockquote class="twitter-tweet"><p lang="en" dir="ltr">A slightly tricky variant: <a href="https://t.co/nxFQfMp2iv">https://t.co/nxFQfMp2iv</a></p>&mdash; Ned Batchelder (@nedbat) <a href="https://twitter.com/nedbat/status/1470062745794101252?ref_src=twsrc%5Etfw">December 12, 2021</a></blockquote> <script async src="https://platform.twitter.com/widgets.js" charset="utf-8"></script>


In [26]:
def cmp(a, b) -> int:
    """ compare two numbers as in python 2

    :returns
        -1 if a < b
        0 if a==b
        1 if a>b
    """
    return (a > b) - (a < b)

This was followed by Robert Smallshire with the comment:

    I use this all the time.


<blockquote class="twitter-tweet"><p lang="da" dir="ltr">&gt;&gt;&gt; def sign_symbol(n):<br>... return (&#39;&#39;, &#39;+&#39;, &#39;-&#39;)[(n &gt; 0) - (n &lt; 0)]<br>... <br>&gt;&gt;&gt; sign_symbol(5)<br>&#39;+&#39;<br>&gt;&gt;&gt; sign_symbol(-5)<br>&#39;-&#39;<br>&gt;&gt;&gt; sign_symbol(0)<br>&#39;&#39;</p>&mdash; Robert Smallshire (@robsmallshire) <a href="https://twitter.com/robsmallshire/status/1470061656373923849?ref_src=twsrc%5Etfw">December 12, 2021</a></blockquote> <script async src="https://platform.twitter.com/widgets.js" charset="utf-8"></script>

What I really like about Robert's function is the joint use of pythons index as a computed value.

Think about this given that most people would write:

In [27]:
def sign_symbol(n):
    if n > 0:
        return '+'
    elif n == 0:
        return ''
    else:
        return '-'

But Roberts use is so much simpler:

In [28]:
def sign_symbol(n):
    return ('','+','-')[(n>0)-(n<0)]

In [29]:
sign_symbol(5), sign_symbol(-5), sign_symbol(0)

('+', '-', '')


## Dictionaries

I've seen other neat tricks like this for dictionaries.

In [30]:
def reorder_alphabetically(**kwargs):
    return {k:v for k,v in zip(sorted(kwargs),sorted(kwargs.values()))}

In [31]:
reorder_alphabetically(a=3,b=1,c=2)

{'a': 1, 'b': 2, 'c': 3}

Accessing nested dictionaries using the dot notation

In [32]:
def get(d, keys):
    focus = d
    for key in keys.split("."):
        focus = focus[key]
    return focus

In [33]:
d = {
    'zero': {"null": {'binary': 0}},
    'one': {"eins": {'binary': 1}, "uno": {'binary': 1}},
    'two': {"zwei": {'binary': 10}, "duo": {'binary': 10}, "to":{'binary': 10}}
}

get(d, "one.eins.binary")

1

In [34]:
get(d, "one.eins")

{'binary': 1}

Get the key of the largest/smallest value in a dictionary

In [35]:
a_dictionary = {"a": 1, "b": 2, "c": 3}

max(a_dictionary, key=a_dictionary.get)

'c'

A two-way fault friendly dictionary

In [36]:
from collections import defaultdict
class TwoWayDict(object):
    def __init__(self):
        self.k_v = {}
        self.v_k = defaultdict(set)

    def __setitem__(self, key, value):
        self.k_v[key]=value
        self.v_k[value].add(key)

    def __getitem__(self, item):
        return self.k_v[item]

    def __delitem__(self, key):
        value = self.k_v.pop(key)
        z = self.v_k[value]
        z.remove(key)
        if not self.v_k[value]:
            del self.v_k[value]

    def get_by_value(self,value):
        """ returns the keys associated with a value"""
        return self.v_k[value]

    def del_by_value(self, value):
        z = self.v_k.pop(value)
        for k in z:
            del self.k_v[k]


In [37]:
dd = TwoWayDict()

In [38]:
dd['a']=10
dd['b']=10
dd['c']=11

In [39]:
dd['a'],dd['b'], dd['c'], dd.get_by_value(10), dd.get_by_value(11)

(10, 10, 11, {'a', 'b'}, {'c'})

In [40]:
del dd['c']

In [41]:
dd.v_k, dd.k_v

(defaultdict(set, {10: {'a', 'b'}}), {'a': 10, 'b': 10})

In [42]:
dd.del_by_value(10)

In [43]:
dd.v_k, dd.k_v

(defaultdict(set, {}), {})